# Import Libraries

In [1]:
import pandas as pd

# Obtain Data

In [3]:
df = pd.read_csv('Data/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


# Scrub Data